### 목표변수와 설명변수
- Subject: 환자
- Goal: Features가 Target인 수술실패여부에 미치는 영향 분석
- Target: '수술실패여부'
- Features: '고혈압여부', '성별', '신부전여부', '연령', '체중', '수술시간'

In [46]:
import pandas as pd

In [47]:
df_ROS = pd.read_csv('../datasets/RecurrenceOfSurgery.csv')
df_ROS[:2]

,Unnamed: 0,환자ID,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,당뇨여부,...,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추이동척도,척추전방위증
0,0,1PT,22.8,3,51.0,0.0,0,0,0,0,...,3,51.6,36.6,14.4,0,-1.01,2048.5,4,Down,0
1,1,2PT,44.9,4,26.0,0.0,0,0,0,0,...,0,40.8,7.2,17.8,0,-1.14,1753.1,4,Up,0


In [48]:
df_ROS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1894 entries, 0 to 1893
Data columns (total 52 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              1894 non-null   int64  
 1   환자ID                    1894 non-null   object 
 2   Large Lymphocyte        1894 non-null   float64
 3   Location of herniation  1894 non-null   int64  
 4   ODI                     462 non-null    float64
 5   가족력                     1843 non-null   float64
 6   간질성폐질환                  1894 non-null   int64  
 7   고혈압여부                   1894 non-null   int64  
 8   과거수술횟수                  1894 non-null   int64  
 9   당뇨여부                    1894 non-null   int64  
 10  말초동맥질환여부                1894 non-null   int64  
 11  빈혈여부                    1894 non-null   int64  
 12  성별                      1894 non-null   int64  
 13  스테로이드치료                 1894 non-null   int64  
 14  신부전여부                   1894 non-null   

### 전처리

In [49]:
df_ROS_select = df_ROS[['수술실패여부', '고혈압여부', '성별', '신부전여부', '연령', '체중', '수술시간']]
df_ROS_select[:2]

,수술실패여부,고혈압여부,성별,신부전여부,연령,체중,수술시간
0,0,0,2,0,66,60.3,68.0
1,0,0,1,0,47,71.7,31.0


#### Scaling & Encoding & Concat

##### - OneHotEncoding

In [50]:
# 범주형 데이터 확인 : '고혈압여부', '성별', '신부전여부'
df_ROS_select['고혈압여부'].value_counts(),df_ROS_select['성별'].value_counts(),df_ROS_select['신부전여부'].value_counts()

(고혈압여부
 0    1646
 1     248
 Name: count, dtype: int64,
 성별
 1    1168
 2     726
 Name: count, dtype: int64,
 신부전여부
 0    1846
 1      48
 Name: count, dtype: int64)

In [51]:
from sklearn.preprocessing import OneHotEncoder

In [52]:
# 범주형 설명변수 OneHotEncoding
oneHotEncoder = OneHotEncoder() # 인스턴스화
oneHotEncoder = oneHotEncoder.fit(df_ROS_select[['고혈압여부', '성별', '신부전여부']])
oneHotEncoder

OneHotEncoder()

In [53]:
oneHotEncoder.categories_

[array([0, 1], dtype=int64),
 array([1, 2], dtype=int64),
 array([0, 1], dtype=int64)]

In [54]:
encoded_data = oneHotEncoder.transform(df_ROS_select[['고혈압여부', '성별', '신부전여부']]).toarray()
encoded_data.shape

(1894, 6)

In [55]:
df_encoded_data = pd.DataFrame(data=encoded_data, columns=oneHotEncoder.get_feature_names_out(['고혈압여부', '성별', '신부전여부']))
df_encoded_data[:2]

,고혈압여부_0,고혈압여부_1,성별_1,성별_2,신부전여부_0,신부전여부_1
0,1.0,0.0,0.0,1.0,1.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0


##### - 병합(Concat)

In [56]:
df_ROS_select= pd.concat([df_ROS_select.reset_index(drop=True), df_encoded_data.reset_index(drop=True)], axis=1)
df_ROS_select[:2]

,수술실패여부,고혈압여부,성별,신부전여부,연령,체중,수술시간,고혈압여부_0,고혈압여부_1,성별_1,성별_2,신부전여부_0,신부전여부_1
0,0,0,2,0,66,60.3,68.0,1.0,0.0,0.0,1.0,1.0,0.0
1,0,0,1,0,47,71.7,31.0,1.0,0.0,1.0,0.0,1.0,0.0


In [57]:
df_ROS_select.shape

(1894, 13)

##### - Scaling

In [58]:
df_ROS_select.columns

Index(['수술실패여부', '고혈압여부', '성별', '신부전여부', '연령', '체중', '수술시간', '고혈압여부_0',
       '고혈압여부_1', '성별_1', '성별_2', '신부전여부_0', '신부전여부_1'],
      dtype='object')

In [59]:
target = df_ROS_select['수술실패여부']
features = df_ROS_select.drop(columns=['수술실패여부', '고혈압여부', '성별', '신부전여부'])

In [60]:
features.columns

Index(['연령', '체중', '수술시간', '고혈압여부_0', '고혈압여부_1', '성별_1', '성별_2', '신부전여부_0',
       '신부전여부_1'],
      dtype='object')

##### - MinMaxScaler

In [61]:
from sklearn.preprocessing import MinMaxScaler

In [62]:
minMaxScaler = MinMaxScaler() #인스턴스화
# features = minMaxSclaer.fit_transform(features)
fit_scaler = minMaxScaler.fit(features)
features_scaler = fit_scaler.transform(features)
features_scaler.shape

(1894, 9)

#### 모델학습, apply()함수를 사용하여 null값 채우기
- null 값 채우기 : '수술시간'

In [63]:
# null값 확인 -> 수술시간 null값 존재
df_ROS_select.isnull().sum()

수술실패여부      0
고혈압여부       0
성별          0
신부전여부       0
연령          0
체중          0
수술시간       54
고혈압여부_0     0
고혈압여부_1     0
성별_1        0
성별_2        0
신부전여부_0     0
신부전여부_1     0
dtype: int64

In [64]:
# null값 삭제 -> null값이 없는 데이터로 모델 학습시키기 위함
df_ROS_select_drop = df_ROS_select.dropna()
df_ROS_select_drop.isnull().sum()

수술실패여부     0
고혈압여부      0
성별         0
신부전여부      0
연령         0
체중         0
수술시간       0
고혈압여부_0    0
고혈압여부_1    0
성별_1       0
성별_2       0
신부전여부_0    0
신부전여부_1    0
dtype: int64

In [65]:
# null값이 없는 데이터로 모델학습 준비
# 1. target : '수술시간', feature: '성별'
target = df_ROS_select_drop[['수술시간']]
feature = df_ROS_select_drop[['성별']]
target.shape, feature.shape

((1840, 1), (1840, 1))

In [66]:
# 2. null값이 없는 데이터와 실제값을 사용하여 회귀모델 훈련
from sklearn.linear_model import LinearRegression
model = LinearRegression() # 인스턴스화(초기화)
model.fit(feature, target) # 모델 훈련

LinearRegression()

In [67]:
# 모델 예측 확인해보기 (type : numpy의 array)
model.predict(feature)

array([[62.40798859],
       [61.85601405],
       [61.85601405],
       ...,
       [61.85601405],
       [61.85601405],
       [62.40798859]])

In [83]:
# apply()
import numpy as np
def convert_notnull(row) :
    if np.isnan(row['수술시간'])  : # 변수 row의 값이 null이라면
        feature = df_ROS_select[['성별']]
        result = model.predict(feature)
        return result[0]
    else :
        return row['수술시간']  # null이 아니면 원래 데이터 값 반환

In [84]:
df_ROS_select['수술시간'] = df_ROS_select.apply(convert_notnull, axis=1)
df_ROS_select['수술시간']

NotFittedError: This DecisionTreeClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [85]:
# apply() 적용 후 null값 확인
df_ROS_select['수술시간'].isnull().sum()

54

In [82]:
df_ROS_select.isnull().sum()

수술실패여부     0
고혈압여부      0
성별         0
신부전여부      0
연령         0
체중         0
수술시간      54
dtype: int64

#### best model 찾기
- hypertuning
- resampling

In [81]:
target = df_ROS_select['수술실패여부']
features = df_ROS_select.drop(columns=['수술실패여부', '고혈압여부', '성별', '신부전여부'])
df_ROS_select.drop(columns=['수술실패여부', '고혈압여부', '성별', '신부전여부']).isnull().sum()

연령       0
체중       0
수술시간    54
dtype: int64

##### - HyperTuning

In [74]:
# 기계학습 알고리즘 호출
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [75]:
# resampling 하기 전
from collections import Counter
Counter(df_ROS_select['수술실패여부'])

Counter({0: 1779, 1: 115})

In [76]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()

In [77]:
from sklearn.metrics import make_scorer, f1_score
scoring = make_scorer(f1_score)
hyper_list = {'max_depth':range(2, 10),
              'min_samples_leaf': range(2, 10),
              'criterion': ['gini','entropy'],
              'class_weight': [None, 'balanced'],
              'min_samples_split': range(2, 10)}
grid_model = GridSearchCV(model, param_grid=hyper_list,
                          scoring=scoring,
                          n_jobs = -1,
                          cv = 5)

##### - resampling
- under sampling : NearMiss

In [78]:
df_ROS_select.isnull().sum()

수술실패여부     0
고혈압여부      0
성별         0
신부전여부      0
연령         0
체중         0
수술시간      54
dtype: int64

In [79]:
from imblearn.under_sampling import NearMiss
from datetime import datetime

In [80]:
sampling_strategy_list = np.arange(0.3, 1.0, 0.05)

for sampling_strategy in sampling_strategy_list :
    print(30*'-','{}'.format(sampling_strategy))
    resampler = NearMiss(sampling_strategy=sampling_strategy)
    features_under_resample, target_under_resample = resampler.fit_resample(features, target)
    print(Counter(target_under_resample))
    features_under_resample_train, features_under_resample_test, target_under_resample_train, target_under_resample_test = train_test_split(features_under_resample, target_under_resample, test_size = 0.3, random_state= 1234)
    print(Counter(target_under_resample_train), Counter(target_under_resample_test))
    # re learning with oversampling datasets
    
    start = datetime.now()  # start date and time

    grid_model.fit(features_under_resample_train, target_under_resample_train)

    end = datetime.now()  # end date and time - set to current date and time

    duration = end - start  # duration as a timedelta object

    print("Duration: {0}".format(duration))
    
    best_model = grid_model.best_estimator_
    target_under_resample_train_pred = best_model.predict(features_under_resample_train)
    target_under_resample_test_pred = best_model.predict(features_under_resample_test)

    print(classification_report(target_under_resample_train, target_under_resample_train_pred))
    print(classification_report(target_under_resample_test, target_under_resample_test_pred))
    print(30*'-')

------------------------------ 0.3


ValueError: Input X contains NaN.
NearMiss does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

#### 정형화

In [ ]:
from sklearn.model_selection import train_test_split
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size = 0.3, random_state = 10)
features_train.shape, features_test.shape, target_train.shape, target_test.shape

#### 모델학습

In [ ]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
model = DecisionTreeClassifier()
# Target이 범주형이므로 Classifier을 사용

In [ ]:
from sklearn.model_selection import GridSearchCV
hyper_params = {'min_samples_leaf' : range(2,5),
               'max_depth' : range(2,5),
               'min_samples_split' : range(2,5)}

#### 예측

In [ ]:
from sklearn.metrics import f1_score, make_scorer
scoring = make_scorer(f1_score)

In [ ]:
grid_search = GridSearchCV(model, param_grid = hyper_params, cv=3, verbose=1, scoring=scoring)
grid_search.fit(features, target)

In [ ]:
grid_search.best_estimator_

In [ ]:
grid_search.best_score_, grid_search.best_params_

# 전처리 전의 정확도(accuracy) : 0.028571428571428574
# 낮은 정확도, 모델이 예측을 잘 수행하지 못함

In [ ]:
best_model = grid_search.best_estimator_
best_model

In [ ]:
target_test_predict = best_model.predict(features_test)
target_test_predict

#### 평가

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(target_test, target_test_predict))

# 전처리 전의 값
#  precision  recall  f1-score   support
#0  0.94      1.00      0.97       516
#1  1.00      0.03      0.05        36

#### 서비스 배포

In [ ]:
# 데이터를 저장하고 불러올 때 매우 유용한 라이브러리
# 클래스 자체를 통째로 파일로저장했다가 그것을 그대로 불러올 수 있음
import pickle

In [ ]:
# MinMaxScaler class pikle
with open('../datasets/RecurrenceOfSurgery_scaling.pkl','wb') as scaling_file :
    pickle.dump(obj=fit_scaler, file=scaling_file)
    pass

In [ ]:
# encoding class pikle
with open('../datasets/RecurrenceOfSurgery_encoding.pkl','wb') as encoding_file :
    pickle.dump(obj=oneHotEncoder, file=encoding_file)
    pass

In [ ]:
# best model pikle
# with open('../datasets/RecurrenceOfSurgery_best_model.pkl','wb') as best_model :
#     pickle.dump(obj=,file=best_model)
#     pass